In [1]:
import json
from pathlib import Path
import math, random
import nltk
import re

import word_category_counter

#Get our data
#Label data based on rating
#Extract features"
#split between train, dev, and test

#Choose classifier 
#Train classifier (on train data)

#Test classifier (on dev data)


In [2]:
#Get wiki (non-joke) data
print("hello")
PATH = Path('data')
files = list(PATH.iterdir())
print(files)
for fname in files:
    if "enwiki" in str(fname) and "gz" not in str(fname):
        wiki_dataset = str(fname)
    if "ROC" in str(fname):
        story_dataset = str(fname)
print(wiki_dataset)
num_lines = sum(1 for line in open(wiki_dataset))
print(num_lines)
wiki_file = open(wiki_dataset,"rb")#.decode("unicode")
abstracts = []
for i in range(num_lines):
    #print(i)
    nextline = wiki_file.readline().decode("utf-8")
    if "bstract" in nextline and "|" not in nextline and "(can)|(may) refer to" and nextline != "__NOTOC__":
        nextline = re.sub("</?abstract>","",nextline)
        nextline = re.sub("}}","",nextline)
        #print(nextline)
        abstracts.append(nextline)
    #if "he" in nextline:
    #    break
print(len(abstracts))        
print("test")
story_file = open(story_dataset)
num_lines = sum(1 for line in open(story_dataset))
print(story_file.readline())
stories = []
for i in range(1,num_lines):
    nextline = story_file.readline()
    cols = nextline.split(",")
    story = " ".join(cols[2:random.randrange(3,7)])
    stories.append(story)
print(story)
notjokes = stories #abstracts

hello
[PosixPath('data/0OYkPK'), PosixPath('data/7R59b1'), PosixPath('data/data'), PosixPath('data/enwiki-20180320-abstract.xml'), PosixPath('data/enwiki-20180320-abstract.xml.gz'), PosixPath('data/fastai'), PosixPath('data/LIWC2007.dic'), PosixPath('data/out'), PosixPath('data/reddit_jokes.json'), PosixPath('data/ROCStories_winter2017 - ROCStories_winter2017.csv'), PosixPath('data/stupidstuff.json'), PosixPath('data/val')]
data/enwiki-20180320-abstract.xml
66316747
3407212
test
storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5

The orange fell from the tree. It hit a girl on the head.


In [3]:
### Get joke data

#standard preprocessing
BOS = 'xbos'  # beginning-of-sentence tag
FLD = 'xfld'  # data field tag
EOJ = 'xeoj'  # end of joke tag

#get jokes:
PATH=Path('data')

files = list(PATH.iterdir())

for fname in files:
    if "eddit" in str(fname):
        reddit_dataset = str(fname)
    if "upid" in str(fname):
        stupid_dataset = str(fname)
reddit_jokes = json.load(open(reddit_dataset))
stupid_jokes = json.load(open(stupid_dataset))

#discard reddit jokes with 0 score:
rated_jokes = [joke for joke in reddit_jokes if joke['score'] > 0]

#regularize to match stupid_jokes:
title_body = [joke['title']+' '+joke['body'] for joke in rated_jokes]

all_jokes = []
for i in range(len(reddit_jokes)):
    r_joke = reddit_jokes[i]
    #|print(r_joke)
    r_joke['rating']=round(math.log(r_joke['score']+random.randrange(1,10))/math.log(10)*5/2, 2)
    if r_joke['rating']>5:
        r_joke['rating']=5
    del r_joke['score'] 
    r_joke['body'] = r_joke['title']+" "+r_joke['body']
    del r_joke['title']
for s_joke in stupid_jokes:
    del s_joke['category']

#combine joke sets:
combined_jokes = reddit_jokes + stupid_jokes

In [4]:
#Group into funny and notFunny sets:

funny_joke_list = []
not_funny_joke_list = []
for joke in combined_jokes:
    if joke["rating"] >= 2.5:
        funny_joke_list.append(joke)
    else:
        not_funny_joke_list.append(joke)
        
(len(funny_joke_list), len(not_funny_joke_list) )

(91652, 106674)

# Feature Extraction:

In [5]:
# get unigrams and bigrams:
stopwords = nltk.corpus.stopwords.words("english")

def normalize(text):
    tokenized_text = []
    tags = []
    for sent in nltk.sent_tokenize(text):
        intermediate = [word for word in nltk.word_tokenize(sent) if (word not in stopwords) and re.search(r"\w", word)]
        for word, pos in nltk.pos_tag(intermediate):
            tokenized_text.append(word.lower())
            tags.append(pos)
    return tokenized_text, tags

def get_ngrams(tokens):
    unigrams = nltk.FreqDist(tokens)
    bigrams = nltk.FreqDist(nltk.bigrams(tokens))
    
    feature_vector = {}
    for token, freq in unigrams.items():
        feature_vector["UNI_%s" %(token)] = float(freq)/unigrams.N()
    for (token1, token2), freq in bigrams.items():
        feature_vector["BI_(%s,%s)" %(token1,token2)] = float(freq)/bigrams.N()        
    return feature_vector
        #"%s ahhhhh! %s" %("sdflks", "sdff")
    
def get_pos(tags):
    unigrams = nltk.FreqDist(tags)
    bigrams = nltk.FreqDist(nltk.bigrams(tags))
    
    feature_vector = {}
    for token, freq in unigrams.items():
        feature_vector["UNIPOS_%s" %(token)] = float(freq)/unigrams.N()
    for (token1, token2), freq in bigrams.items():
        feature_vector["BIPOS_(%s,%s)" %(token1,token2)] = float(freq)/bigrams.N()        
    return feature_vector

def get_liwc_features(tokens):
    """
    Adds all possible LIWC derived feature

    :param words:
    :return:
    """
    text = u" ".join(tokens)
    liwc_cat = ['Total Function Words',
                'Total Pronouns',
                'Personal Pronouns',
                'First Person Singular',
                'First Person Plural',
                'Second Person',
                'Third Person Singular',
                'Third Person Plural',
                ' Impersonal Pronouns',
                'Articles',
                'Common Verbs',
                'Auxiliary Verbs',
                'Past Tense',
                'Present Tense',
                'Future Tense',
                'Adverbs',
                'Prepositions',
                'Conjunctions',
                'Negations',
                'Quantifiers',
                'Number',
                'Swear Words',
                'Social Processes',
                'Family',
                'Friends',
                'Humans',
                'Affective Processes',
                'Positive Emotion',
                'Negative Emotion',
                'Anxiety',
                'Anger',
                'Sadness',
                'Cognitive Processes',
                'Insight',
                'Causation',
                'Discrepancy',
                'Tentative',
                'Certainty',
                'Inhibition',
                'Inclusive',
                'Exclusive',
                'Perceptual Processes',
                'See',
                'Hear',
                'Feel',
                'Biological Processes',
                'Body',
                'Health',
                'Sexual',
                'Ingestion',
                'Relativity',
                'Motion',
                'Space',
                'Time',
                'Work',
                'Achievement',
                'Leisure',
                'Home',
                'Money',
                'Religion',
                'Death',
                'Assent',
                'Nonfluencies',
                'Fillers',
                'Total first person',
                'Total third person',
                'Positive feelings',
                'Optimism and energy',
                'Communication',
                'Other references to people',
                'Up',
                'Down',
                'Occupation',
                'School',
                'Sports',
                'TV',
                'Music',
                'Metaphysical issues',
                'Physical states and functions',
                'Sleeping',
                'Grooming']

    feature_vectors = {}
    liwc_scores = word_category_counter.score_text(text)

    negative_score = liwc_scores["Negative Emotion"]
    positive_score = liwc_scores["Positive Emotion"]

    if positive_score > negative_score:
        feature_vectors["liwc:positive"] = 1
    else:
        feature_vectors["liwc:negative"] = 1

    for cat in liwc_cat:
        if cat in liwc_scores:
            label = cat.lower().replace(" ", "_")
            feature_vectors["liwc_%s" %label] = liwc_scores[cat]

    return feature_vectors

In [6]:
joke_feature_tuples = []
set_size= 4000
for joke in combined_jokes[:set_size]:
    tokens, tags = normalize(joke["body"])
    joke_feature_tuples.append(({**get_ngrams(tokens), **get_pos(tags), **get_liwc_features(tokens)},"joke"))
    
notjoke_feature_tuples = []
for abstract in notjokes[:set_size]:
    tokens, tags = normalize(abstract)
    notjoke_feature_tuples.append(({**get_ngrams(tokens), **get_pos(tags), **get_liwc_features(tokens)},"not_joke"))

In [7]:
notjoke_feature_tuples[0]

({'BIPOS_(JJ,NN)': 0.05263157894736842,
  'BIPOS_(MD,VBG)': 0.05263157894736842,
  'BIPOS_(NN,NN)': 0.15789473684210525,
  'BIPOS_(NN,PRP)': 0.05263157894736842,
  'BIPOS_(NN,RB)': 0.10526315789473684,
  'BIPOS_(NNP,VBD)': 0.05263157894736842,
  'BIPOS_(NNS,VB)': 0.05263157894736842,
  'BIPOS_(PRP,VBD)': 0.10526315789473684,
  'BIPOS_(RB,PRP)': 0.05263157894736842,
  'BIPOS_(VB,NN)': 0.05263157894736842,
  'BIPOS_(VBD,MD)': 0.05263157894736842,
  'BIPOS_(VBD,NNS)': 0.05263157894736842,
  'BIPOS_(VBD,VBN)': 0.05263157894736842,
  'BIPOS_(VBG,JJ)': 0.05263157894736842,
  'BIPOS_(VBN,NN)': 0.05263157894736842,
  "BI_('d,eating)": 0.05263157894736842,
  'BI_(david,noticed)': 0.05263157894736842,
  'BI_(eating,much)': 0.05263157894736842,
  'BI_(examined,habits)': 0.05263157894736842,
  'BI_(fast,food)': 0.05263157894736842,
  'BI_(figure,reason)': 0.05263157894736842,
  'BI_(food,lately)': 0.05263157894736842,
  'BI_(habits,try)': 0.05263157894736842,
  'BI_(he,examined)': 0.05263157894736

### Partitioning

In [8]:
division_size = int(set_size*4/5)
train = joke_feature_tuples[:division_size]+notjoke_feature_tuples[:division_size]
dev = joke_feature_tuples[division_size:]+notjoke_feature_tuples[division_size:]

# Training:

In [9]:
classifier = nltk.classify.NaiveBayesClassifier.train(train)

In [10]:
accuracy = nltk.classify.accuracy(classifier, dev)
accuracy

0.850625

In [11]:
features_only = []
labels_only = []
for vector, label in dev:
    features_only.append(vector)
    labels_only.append(label)
    

predicted_labels = classifier.classify_many(features_only)

confusion_matrix = nltk.ConfusionMatrix(labels_only, predicted_labels)
print(confusion_matrix)

         |       n |
         |       o |
         |       t |
         |       _ |
         |   j   j |
         |   o   o |
         |   k   k |
         |   e   e |
---------+---------+
    joke |<629>171 |
not_joke |  68<732>|
---------+---------+
(row = reference; col = test)



In [12]:
test = "There was an orange on the tree. Someone was standing under the tree. Here is another sentence. And another one. And here is a fifth."
test = "Why is it unknown on how pterodactyls urinate especially during flight?"
test = "i hate how you can't stop for 7 years. he flings his dibty-out to sitting at a bell and asked the faster ''we started carrying and utteringal at a heary at this first 'could you refresh?' the man comes back to his bought and scared as it's only day, 'i'm walking on by because. the battle runs farted to the entire lives of arm off a body where he refugeed to be from his chair woman as heaven, but she sat up was water.one of it and he finds the ownerhip showing up. this is an old man cated i but start coasination"
tests = [
"what do you tricking without the money?\"\"there's able together.\"there's hairy rounds the hundered, and.a kid unturns the 'other rednecked.",

"into a bar, i think then. we togl indeader drinking it fur slowly bread with not to has an alleday to tjumble between. after the doors and it wouldn't win? an drants 10 hmph.",

"what do roly sure it decides her on the bills out of the 7 heevy in lim sorry-kiblive selfripal day and where does pactive a section, and he was grabbing the hospit faster? it's good gare anguagas.  you can't snow likes.son: ih",

"blon do younmation? \"the came at my privator. light.> i asking, \"land,\" cled day, \"why did land why a life wond-r/zep_aggers/k2k came - thatic-coup.immatterbary.one decity crazh of seexic ange s next \"ca4",

"a joke the mouely, he sees on the natted intral remember for supported the looking for a mpc.the things, \"yes, i wont he said than why did.",

"a subjle to a merger... ..   i inkter by cne. \"it's usually barely been explaining the foresian for you.one day, s",

"Whits, says a xear, wold what What the man to hard there heran alinuts to the sistel third the Conks t",

"Wha love now some he lower told can no head words about it to a brotheretter commite. So you want the",

"What’s working and a store in are of was got in more by the off the most around bring back as in a st",

"Where you have he sitting he was a nervose happened. Why can on the cather hold a bor?",

"Where a monkey can! Why was bikine for the hot do dubbing and all devatrist sex? Because a moment. He",

"What’s a some from a turn with there? The bull in But I’ll don are ask. He couldn’t better with a ball",

"i hate how you can't stop for 7 years. he flings his dibty-out to sitting at a bell and asked the faster \"\"we started carrying and utteringal at a heary at this first \"could you refresh?\" the man comes back to his bought and scared as it's only day, \"i'm walking on by because. the battle runs farted to the entire lives of arm off a body where he refugeed to be from his chair woman as heaven, but she sat up was water.one of it and he finds the ownerhip showing up. this is an old man cated i but start coasination",

"i hate how you can't st.  \"hey, so no, are an expensive breaked the temperature to left, he makes the lead, and the swimming rules in a back of holds. and the inside to great as and old : i just make it off her healfuses. in the wines were logged by a good shop-across and thought more really englishman, thought he heard enough trishung. she giving the hope to be fruit as she was a new debuted pretty drinking and puts it.but they can get it on easy!",

"i hate how you can't see swet president and asked, \"what does it?\”",

"i hate how you can't stupid dick?\"i",

"why women need legs.",

"why women need legse fishing one of his back and starts sboft ingurily.from bay. \"you're everyone were cartain.",

"why women need legsy arms watting the attraction of the guy gets not try. **apain skin on eyem!",

"why women need legs for a few million loop, and this were holding its time. replied, \“'jae, so it's jewish tases",

"what do you think how i fuck us \" after god doing jack hole trump are hitting us. say, and to look down!"

]
for test in tests:
    tokens, tags = normalize(test)
    print(classifier.classify({**get_ngrams(tokens), **get_pos(tags), **get_liwc_features(tokens)}))

not_joke
not_joke
not_joke
not_joke
joke
joke
joke
joke
joke
joke
joke
joke
joke
joke
joke
joke
not_joke
joke
joke
joke
joke


# Things to try:

Word embeddings

Binning